In [1]:
import os
import numpy as np
import numpy.matlib
from scipy.io import wavfile
from pydub import AudioSegment

In [2]:
files = os.listdir("/home/andrea/Scrivania/MIVIA_ROAD_DB1/splitted_dataset/")
for i,f in enumerate(files):
    files[i] = "/home/andrea/Scrivania/MIVIA_ROAD_DB1/splitted_dataset/"+f
files = sorted(files)

In [3]:
maxlen = 0
for f in files:
    s = AudioSegment.from_file(f)
    samples = s.get_array_of_samples()
    if len(samples) > maxlen:
        maxlen = len(samples)

In [3]:
def fill(file, maxlength=264256):
    if len(file) < maxlength:
        complete = np.zeros((maxlength,), dtype=np.int16)
        filled = 0
        while filled < maxlength:
            if filled+len(file) < maxlength:
                complete[filled:filled+len(file)] = file
            else:
                complete[filled:] = file[:maxlength-filled]
            filled+=len(file)
        return  complete
    return file

In [43]:
noises = []
def create_random_bg_noises():
    import random
    for _ in range(15):
        audios = []
        for _ in range(3):
            chosen = None
            num = str(random.randint(400, len(files)))
            chosen = searchFile(num)
            audios.append(chosen)
        base = AudioSegment.from_file(audios[0])
        base2 = base.overlay(AudioSegment.from_file(audios[1]))
        base3 = base2.overlay(AudioSegment.from_file(audios[2]))
        complete = fill(np.asarray(base3.get_array_of_samples()))
        noises.append(AudioSegment(complete.tobytes(),
                      frame_rate=base.frame_rate,
                      sample_width=base.frame_width,
                      channels=1))
create_random_bg_noises()

In [5]:
noises = []
def full_noises():
    for i in range(400,742):
        noise = AudioSegment.from_file(searchFile(str(i)))
        complete = fill(np.asarray(noise.get_array_of_samples()))
        noises.append(AudioSegment(complete.tobytes(),
                      frame_rate=noise.frame_rate,
                      sample_width=noise.frame_width,
                      channels=1))
full_noises()

In [6]:
def add_noise_and_normalize_length(file, noise, maxlength=264256):
    new_audio = np.zeros((maxlength,), dtype=np.int16)
    stest = AudioSegment.from_file(file)
    samples = np.asarray(stest.get_array_of_samples())
    new_audio[:len(samples)] = samples
    s2 =  AudioSegment(new_audio.tobytes(),
                      frame_rate=stest.frame_rate,
                      sample_width=stest.frame_width,
                      channels=1)
    return s2.overlay(noise)

In [19]:
s = AudioSegment.from_file(files[0])
samples = np.asarray(s.get_array_of_samples())
filled = fill(samples)
s2 =  AudioSegment(filled.tobytes(),
                      frame_rate=s.frame_rate,
                      sample_width=s.frame_width,
                      channels=1)
s2.export("/home/andrea/Scrivania/MIVIA_ROAD_DB1/s.wav", format='wav')

<_io.BufferedRandom name='/home/andrea/Scrivania/MIVIA_ROAD_DB1/s.wav'>

In [4]:

def searchFile(name):
    for f in files:
        splitted = os.path.split(f)
        num, ext = splitted[-1].split(".")
        if num == name:
            return f

In [8]:
def mix(audio1, audio2):
    sound1 = AudioSegment.from_file(audio1)
    sound2 = AudioSegment.from_file(audio2)
    return sound1.overlay(sound2)

In [7]:
noised_files = []
for i in range(400):
    import random
    clear = searchFile(str(i))
    j = random.randint(0,341)
    noised_files.append(add_noise_and_normalize_length(clear, noises[j]))

In [18]:
for i in range(400):
    import random
    clear = noised_files[i]
    sample = clear.get_array_of_samples()
    sample = sample[:264256]
    noised_files[i] =  AudioSegment(sample.tobytes(),
                      frame_rate=clear.frame_rate,
                      sample_width=clear.frame_width,
                      channels=1)

    print(len(noised_files[i].get_array_of_samples()))

264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256
264256

In [19]:
classes = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/classes.npy")
binary = []
for i, n in enumerate(noised_files):
    if classes[i] != 0:
        binary.append(1)
    n.export("/home/andrea/Scrivania/MIVIA_ROAD_DB1/noised_dataset/"+str(i)+".wav", format='wav')
for j in range(400,742):
    binary.append(0)
    noises[j-400].export("/home/andrea/Scrivania/MIVIA_ROAD_DB1/noised_dataset/"+str(j)+".wav", format='wav')
np.save("/home/andrea/Scrivania/MIVIA_ROAD_DB1/bin_classes.npy", np.asarray(binary))

In [ ]:
s1 = AudioSegment.from_file(files[0])
s2 = AudioSegment.from_file(files[1])
s3 = mix(searchFile("0"), searchFile("1"))

In [5]:
files[730]

'/home/andrea/Scrivania/MIVIA_ROAD_DB1/splitted_dataset/89.wav'

In [ ]:
np_files = np.ndarray((len(files),))

In [14]:
events, bg = [],[]
np_classes = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/classes.npy")

for n in range(400):
    events.append(searchFile(str(n)))
for m in range(400,742):
    bg.append(searchFile(str(m)))

In [21]:
classes = []
counter = 0
for i, e in enumerate(events):
    for j, b in enumerate(bg):
        classes.append(np_classes[i])
        new = mix(e,b)
        new.export("/mnt/3206BFFF191E7F85/extended_dataset/"+str(counter)+".wav", format='wav')
        counter+=1
for i, e in enumerate(bg):
    for j, b in enumerate(bg):
        classes.append(0)
        new = mix(e,b)
        new.export("/mnt/3206BFFF191E7F85/extended_dataset/"+str(counter)+".wav", format='wav')
        counter+=1
classes_np = np.asarray(classes)

In [20]:
np.save("/home/andrea/Scrivania/MIVIA_ROAD_DB1/splitted_dataset/classes_ext.npy", classes_np)

In [32]:
files_list = os.listdir("/home/andrea/Scrivania/MIVIA_ROAD_DB1/noised_dataset/")
for i,f in enumerate(files_list):
    files_list[i] = "/home/andrea/Scrivania/MIVIA_ROAD_DB1/noised_dataset/"+f
samples = []
def searchNoisedFile(name):
    for f in files_list:
        splitted = os.path.split(f)
        num, ext = splitted[-1].split(".")
        if num == name:
            return f
for i in range(742):
    samples.append(np.asarray(AudioSegment.from_file(searchNoisedFile(str(i))).get_array_of_samples()))

In [33]:
dataset = np.zeros((len(samples),16,16516), dtype=np.int16)

In [34]:
step = 16516
for i, sample in enumerate(samples):
    for j in range(16):
        dataset[i,j,:] = sample[j*step:(j+1)*step]

In [35]:
np.save("/home/andrea/Scrivania/MIVIA_ROAD_DB1/dataset.npy", dataset)